In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
path = "/home/herb/Documents/private/MLE_CV.pdf"
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(path)
documents = loader.load()
print(len(documents))

1


In [5]:
print(documents[0].metadata)
print(documents[0].page_content[:200])

{'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-07-14T19:25:48+00:00', 'author': '', 'keywords': '', 'moddate': '2025-07-14T19:25:48+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/home/herb/Documents/private/MLE_CV.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}
Matvey Kamenev
+7 (922) 244-99-77 | mvkamenev@edu.hse.ru | t.me/rumatstone | github.com/herbitovich
Education
Higher School of Economics Saint-Petersburg, Russia
Bachelor of Applied Data Analysis and 


In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20, add_start_index=True)
all_splits = text_splitter.split_documents(documents)
print(len(all_splits))
print(all_splits[0])

36
page_content='Matvey Kamenev
+7 (922) 244-99-77 | mvkamenev@edu.hse.ru | t.me/rumatstone | github.com/herbitovich' metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-07-14T19:25:48+00:00', 'author': '', 'keywords': '', 'moddate': '2025-07-14T19:25:48+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/home/herb/Documents/private/MLE_CV.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'start_index': 0}


In [10]:
from langchain_mistralai import MistralAIEmbeddings
embeddings = MistralAIEmbeddings(model="mistral-embed")

/home/herb/Documents/programming/langchain_learning/langchain_venv/lib64/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/herb/Documents/programming/langchain_learning/langchain_venv/lib64/python3.13/site-packages/langchain_mistralai/embeddings.py:186: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [15]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [16]:
vector_store.add_documents(all_splits)

['29cfa493-915c-47f5-a4a7-4cd785f08e5d',
 'd5b9093d-bff3-40c7-b51f-72de05995c89',
 '0e3af806-b45d-4b4c-994f-b1c5111e54e2',
 'f873570a-1c84-464a-8fe7-ea1be8936413',
 'accf7df6-10e4-467e-a73d-8d45c95554a1',
 '2d7bf748-23d5-4f40-a27a-b349f5762f7f',
 '31f765ac-39af-48b4-8ef5-75e696a452bc',
 '488dbf33-3410-40e4-8459-018715290c3b',
 '2b50cbfe-af8a-49a6-8b45-753b5565f09b',
 'b4e6f67c-626b-4285-8873-af0e2fdeef6e',
 '9541c990-a6bf-4ef7-83f6-622485b49df1',
 '58ecc5ee-888c-4de5-91cc-5ad63340685e',
 'ea624588-d9cf-4790-a942-6ad7a67f0f17',
 'c41cde35-f6c7-4fcb-9851-669f3a5dfcb6',
 '1cbcd196-c94d-4716-a47b-6b5e15471723',
 'd277c9e9-b32f-491c-887f-e5a10b6648e3',
 '52be86ca-7ca0-4946-9573-4e851c2a7393',
 '6387f4cd-f918-47d0-9ffb-c81909cc3370',
 '68c59f7d-14db-473f-9321-49bdc0dc356c',
 '7c1cc549-8357-422d-80a7-45d09b09484c',
 'c472003a-dc55-4c26-bbfe-afce235e276e',
 '19cabce1-5b11-452e-9632-2eae1939bb31',
 '6a1bcb8b-8715-4cab-833b-78be6aed0b7e',
 'be08fc8e-4297-4c82-a847-f55cc3b9f1b2',
 '3d764691-15d5-

In [19]:
results = vector_store.similarity_search(
    "What frameworks is Matvey profficient in?",
    k = 1
)
results[0].page_content

'Frameworks: PyTorch, TensorFlow, FastAI, FastAPI, Django, Django REST, Flask'

In [35]:
results = vector_store.similarity_search_with_score(
    "What framework did Matvey use in web development?",
    k = 3
)
for res in results:
    doc, score = res
    print("RES")
    print(doc.page_content)
    print(score)

RES
• Developed various web applications using the Django w/ PostgreSQL stack
0.7942863968081733
RES
Frameworks: PyTorch, TensorFlow, FastAI, FastAPI, Django, Django REST, Flask
0.7626423571714511
RES
Matvey Kamenev
+7 (922) 244-99-77 | mvkamenev@edu.hse.ru | t.me/rumatstone | github.com/herbitovich
0.739381778239946


In [36]:
from langchain_core.runnables import chain

@chain
def retriever(query):
    return vector_store.similarity_search(query, k=1)

retriever.batch([
    "What frameworks is Matvey profficient in?",
    "What framework did Matvey use in web development?",
])

[[Document(id='f53f4944-4d8e-4330-bf04-958ecc8b4325', metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-07-14T19:25:48+00:00', 'author': '', 'keywords': '', 'moddate': '2025-07-14T19:25:48+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/home/herb/Documents/private/MLE_CV.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'start_index': 2229}, page_content='Frameworks: PyTorch, TensorFlow, FastAI, FastAPI, Django, Django REST, Flask')],
 [Document(id='7c1cc549-8357-422d-80a7-45d09b09484c', metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-07-14T19:25:48+00:00', 'author': '', 'keywords': '', 'moddate': '2025-07-14T19:25:48+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', '

In [38]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)

retriever.batch([
    "What frameworks is Matvey profficient in?",
    "What framework did Matvey use in web development?",
])

[[Document(id='f53f4944-4d8e-4330-bf04-958ecc8b4325', metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-07-14T19:25:48+00:00', 'author': '', 'keywords': '', 'moddate': '2025-07-14T19:25:48+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/home/herb/Documents/private/MLE_CV.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'start_index': 2229}, page_content='Frameworks: PyTorch, TensorFlow, FastAI, FastAPI, Django, Django REST, Flask')],
 [Document(id='7c1cc549-8357-422d-80a7-45d09b09484c', metadata={'producer': 'pdfTeX-1.40.26', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-07-14T19:25:48+00:00', 'author': '', 'keywords': '', 'moddate': '2025-07-14T19:25:48+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) kpathsea version 6.4.0', 'subject': '', 'title': '', '